In [1]:
from openbb import obb
obb.user.preferences.output_type = "dataframe"

In [6]:
chains = obb.derivatives.options.chains(symbol="SPY")
chains.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10786 entries, 0 to 10785
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   underlying_symbol   10786 non-null  object 
 1   underlying_price    10786 non-null  float64
 2   contract_symbol     10786 non-null  object 
 3   expiration          10786 non-null  object 
 4   dte                 10786 non-null  int64  
 5   strike              10786 non-null  float64
 6   option_type         10786 non-null  object 
 7   open_interest       10786 non-null  int64  
 8   volume              10786 non-null  int64  
 9   theoretical_price   10786 non-null  float64
 10  last_trade_price    10786 non-null  float64
 11  last_trade_time     8520 non-null   object 
 12  tick                10786 non-null  object 
 13  bid                 10786 non-null  float64
 14  bid_size            10786 non-null  int64  
 15  ask                 10786 non-null  float64
 16  ask_

In [ ]:
data = obb.equity.price.historical(
    symbol="SPY251220C00550000",
)[["close", "volume"]]

OpenBBError: 
[Unexpected Error] -> ContentTypeError -> 403, message='Attempt to decode JSON with unexpected mimetype: application/xml', url='https://cdn.cboe.com/api/global/delayed_quotes/charts/historical/SPY251220C00550000.json'

In [7]:
import yfinance as yf

option_symbol = "SPY251220C00550000"

# Step 1 — Parse the OCC format manually
underlying = option_symbol[:3]  # SPY
year = "20" + option_symbol[3:5]  # 25 -> 2025
month = option_symbol[5:7]        # 12
day = option_symbol[7:9]          # 20
call_put = option_symbol[9]       # C
strike_raw = option_symbol[10:]   # 00550000
strike = float(strike_raw) / 1000 # strike encoded x1000

expiration_date = f"{year}-{month}-{day}"

print("Parsed Option Info:")
print(underlying, expiration_date, call_put, strike)

# Step 2 — Query option chain
ticker = yf.Ticker(underlying)
chain = ticker.option_chain(expiration_date)

# Get correct chain
calls_df = chain.calls if call_put == "C" else chain.puts

# Step 3 — Locate correct contract
row = calls_df.loc[calls_df["strike"] == strike].iloc[0]

contract_ticker = row["contractSymbol"]
print("Matched contract symbol:", contract_ticker)

# Step 4 — Download historical data
option_hist = yf.download(contract_ticker, start="2024-01-01")

df = option_hist[["Close", "Volume"]].rename(columns={
    "Close": "close",
    "Volume": "volume"
})

print(df.head())

Parsed Option Info:
SPY 2025-12-20 C 550.0


TypeError: CurlStreamResponse._decode() got an unexpected keyword argument 'max_length'